In [1]:
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, TimeSeriesSplit

In [3]:
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
f = pd.read_csv('C:\\beijing_engineered.csv')
df = pd.DataFrame(f)

In [5]:
df['date'] = pd.to_datetime(df.date)

In [6]:
# Setting DateTime index
df['date'] = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43799 entries, 2010-01-02 00:00:00 to 2014-12-31 22:00:00
Data columns (total 49 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pm25         43799 non-null  int64  
 1   dewp         43799 non-null  int64  
 2   temp         43799 non-null  int64  
 3   cws          43799 non-null  float64
 4   target       43799 non-null  float64
 5   cwkend       43799 non-null  float64
 6   cbwd_1       43799 non-null  int64  
 7   cbwd_2       43799 non-null  int64  
 8   cbwd_3       43799 non-null  int64  
 9   mth_sin      43799 non-null  float64
 10  mth_cos      43799 non-null  float64
 11  hour_sin     43799 non-null  float64
 12  hour_cos     43799 non-null  float64
 13  hour_num     43799 non-null  float64
 14  mth_num      43799 non-null  float64
 15  cmonth_2.0   43799 non-null  int64  
 16  cmonth_3.0   43799 non-null  int64  
 17  cmonth_4.0   43799 non-null  int64  
 18  cmonth_5.0 

In [8]:
df[['pm25', 'dewp', 'temp', 'cws', 'target', 'hour_num', 'mth_num']]

,pm25,dewp,temp,cws,target,hour_num,mth_num
date,,,,,,,
2010-01-02 00:00:00,129,-16,-4,1.79,148.0,1.0,1.0
2010-01-02 01:00:00,148,-15,-4,2.68,159.0,2.0,1.0
2010-01-02 02:00:00,159,-11,-5,3.57,181.0,3.0,1.0
2010-01-02 03:00:00,181,-7,-5,5.36,138.0,4.0,1.0
2010-01-02 04:00:00,138,-7,-5,6.25,109.0,5.0,1.0
...,...,...,...,...,...,...,...
2014-12-31 18:00:00,10,-22,-2,226.16,8.0,19.0,12.0
2014-12-31 19:00:00,8,-23,-2,231.97,10.0,20.0,12.0
2014-12-31 20:00:00,10,-22,-3,237.78,10.0,21.0,12.0


In [9]:
# Change the integer variables to int32 type to reduce memory usage
df[['pm25', 'dewp', 'temp', 'target', 'hour_num', 'mth_num']] = \
    df[['pm25', 'dewp', 'temp', 'target', 'hour_num', 'mth_num']].astype(np.int32)

In [10]:
# Change the dummy variables to uint8 type to reduce memory usage
df[['cmonth_2.0', 'cmonth_3.0', 'cmonth_4.0', 'cmonth_5.0', 'cmonth_6.0', 'cmonth_7.0', 'cmonth_8.0', 'cmonth_9.0',
    'cmonth_10.0', 'cmonth_11.0', 'cmonth_12.0', 'chour_1.0', 'chour_2.0', 'chour_3.0', 'chour_4.0', 'chour_5.0',
    'chour_6.0', 'chour_7.0', 'chour_8.0', 'chour_9.0', 'chour_10.0', 'chour_11.0', 'chour_12.0', 'chour_13.0', 
    'chour_14.0', 'chour_15.0', 'chour_16.0', 'chour_17.0', 'chour_18.0', 'chour_19.0', 'chour_20.0', 'chour_21.0',
    'chour_22.0', 'chour_23.0', 'cwkend', 'cbwd_1', 'cbwd_2', 'cbwd_3']] = \
    df[['cmonth_2.0', 'cmonth_3.0', 'cmonth_4.0', 'cmonth_5.0', 'cmonth_6.0', 'cmonth_7.0', 'cmonth_8.0', 
        'cmonth_9.0', 'cmonth_10.0', 'cmonth_11.0', 'cmonth_12.0', 'chour_1.0', 'chour_2.0', 'chour_3.0',
        'chour_4.0', 'chour_5.0', 'chour_6.0', 'chour_7.0', 'chour_8.0', 'chour_9.0', 'chour_10.0', 'chour_11.0',
        'chour_12.0', 'chour_13.0', 'chour_14.0', 'chour_15.0', 'chour_16.0', 'chour_17.0', 'chour_18.0',
        'chour_19.0', 'chour_20.0', 'chour_21.0', 'chour_22.0', 'chour_23.0', 'cwkend', 'cbwd_1', 'cbwd_2', 
        'cbwd_3']].astype(np.uint8)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43799 entries, 2010-01-02 00:00:00 to 2014-12-31 22:00:00
Data columns (total 49 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pm25         43799 non-null  int32  
 1   dewp         43799 non-null  int32  
 2   temp         43799 non-null  int32  
 3   cws          43799 non-null  float64
 4   target       43799 non-null  int32  
 5   cwkend       43799 non-null  uint8  
 6   cbwd_1       43799 non-null  uint8  
 7   cbwd_2       43799 non-null  uint8  
 8   cbwd_3       43799 non-null  uint8  
 9   mth_sin      43799 non-null  float64
 10  mth_cos      43799 non-null  float64
 11  hour_sin     43799 non-null  float64
 12  hour_cos     43799 non-null  float64
 13  hour_num     43799 non-null  int32  
 14  mth_num      43799 non-null  int32  
 15  cmonth_2.0   43799 non-null  uint8  
 16  cmonth_3.0   43799 non-null  uint8  
 17  cmonth_4.0   43799 non-null  uint8  
 18  cmonth_5.0 

# Create Cyclic Seasonality Dataset

In [12]:
df_cyc = df.drop(['cmonth_2.0', 'cmonth_3.0', 'cmonth_4.0', 'cmonth_5.0', 'cmonth_6.0', 'cmonth_7.0', 
                  'cmonth_8.0', 'cmonth_9.0', 'cmonth_10.0', 'cmonth_11.0', 'cmonth_12.0', 'chour_1.0', 
                  'chour_2.0', 'chour_3.0', 'chour_4.0', 'chour_5.0', 'chour_6.0', 'chour_7.0', 
                  'chour_8.0', 'chour_9.0', 'chour_10.0', 'chour_11.0', 'chour_12.0', 'chour_13.0', 
                  'chour_14.0', 'chour_15.0', 'chour_16.0', 'chour_17.0', 'chour_18.0', 'chour_19.0', 
                  'chour_20.0', 'chour_21.0', 'chour_22.0', 'chour_23.0', 'mth_num', 'hour_num'], axis=1)

# Preparing Data for Optimization

### Train-Test Split

In [13]:
# Set test set at 20% of data
sample = int(len(df)*0.2)

In [14]:
sample

8759

In [15]:
df_cyct = df_cyc.iloc[-sample:]
df_cyc = df_cyc.iloc[:-sample]

In [16]:
df_cyc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35040 entries, 2010-01-02 00:00:00 to 2013-12-31 23:00:00
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pm25      35040 non-null  int32  
 1   dewp      35040 non-null  int32  
 2   temp      35040 non-null  int32  
 3   cws       35040 non-null  float64
 4   target    35040 non-null  int32  
 5   cwkend    35040 non-null  uint8  
 6   cbwd_1    35040 non-null  uint8  
 7   cbwd_2    35040 non-null  uint8  
 8   cbwd_3    35040 non-null  uint8  
 9   mth_sin   35040 non-null  float64
 10  mth_cos   35040 non-null  float64
 11  hour_sin  35040 non-null  float64
 12  hour_cos  35040 non-null  float64
dtypes: float64(5), int32(4), uint8(4)
memory usage: 2.3 MB


In [17]:
df_cyct.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8759 entries, 2014-01-01 00:00:00 to 2014-12-31 22:00:00
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pm25      8759 non-null   int32  
 1   dewp      8759 non-null   int32  
 2   temp      8759 non-null   int32  
 3   cws       8759 non-null   float64
 4   target    8759 non-null   int32  
 5   cwkend    8759 non-null   uint8  
 6   cbwd_1    8759 non-null   uint8  
 7   cbwd_2    8759 non-null   uint8  
 8   cbwd_3    8759 non-null   uint8  
 9   mth_sin   8759 non-null   float64
 10  mth_cos   8759 non-null   float64
 11  hour_sin  8759 non-null   float64
 12  hour_cos  8759 non-null   float64
dtypes: float64(5), int32(4), uint8(4)
memory usage: 581.7 KB


In [18]:
# Target variables, which will be the SAME for all the various datasets
y_cyct = df_cyct.pop('target')
y_cyc = df_cyc.pop('target')

### Scaling data

In [19]:
# Using MinMaxScaler due to the comparison with the dummy seasonality treatment
scaler = MinMaxScaler()

In [20]:
# Scaling the cyclic dataset
X_cyc = scaler.fit_transform(df_cyc)
X_cyct = scaler.transform(df_cyct)

In [21]:
#Make an inner and outer validation scheme for Nested Cross-Validation
time_split = TimeSeriesSplit(n_splits=3)

# Randomized Search CV

Optimization on the **cyclic seasonality** dataset

In [22]:
# Lasso Regression
lasso = Lasso(random_state=11)

las_dist = {'fit_intercept': [1, 0],
            'alpha': uniform(0.001, 2)}


In [23]:
rs_las = RandomizedSearchCV(lasso, las_dist, cv=time_split, scoring='neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [24]:
rs_las.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Lasso(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F931910>,
                                        'fit_intercept': [1, 0]},
                   scoring='neg_mean_squared_error', verbose=1)

In [25]:
# Ridge Regression
ridge = Ridge(random_state=11)

rdg_dist = {'fit_intercept': [1, 0],
            'solver': ['lsqr', 'sag', 'cholesky'],
            'alpha': uniform(0.001, 2)}


In [26]:
rs_rdg = RandomizedSearchCV(ridge, rdg_dist, cv=time_split, scoring='neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [27]:
rs_rdg.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Ridge(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F1C27C0>,
                                        'fit_intercept': [1, 0],
                                        'solver': ['lsqr', 'sag', 'cholesky']},
                   scoring='neg_mean_squared_error', verbose=1)

In [28]:
# Random Forest
rf = RandomForestRegressor(random_state=11)

rf_dist = {'n_estimators': randint(50, 500),
           'min_samples_split': randint(2, 9),
           'max_features': ['auto', 'log2', 'sqrt']}


In [29]:
rs_rf = RandomizedSearchCV(rf, rf_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                           n_jobs=-1, verbose=1)

In [30]:
rs_rf.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=RandomForestRegressor(random_state=11), n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2',
                                                         'sqrt'],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F6F8310>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F47CE80>},
                   scoring='neg_mean_squared_error', verbose=1)

In [31]:
# XG Boost Regressor
xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=11)

xgb_dist = {'n_estimators': randint(50, 500),
            'subsample': [0.5, 0.7, 1],
            'eta': uniform(0.05, 1),
            'gamma': randint(0, 300)}


In [32]:
rs_xgb = RandomizedSearchCV(xgb, xgb_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [33]:
rs_xgb.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_w...
                                          verbosity=None),
                   n_jobs=-1,
                   param_distributions={'eta': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F5329D0>,
                                        'gamma': <scipy.stats._

In [34]:
# Support Vector Regressor
svm = SVR()

svm_dist = {'kernel': ['rbf', 'linear', 'poly'],
            'gamma': ['scale', 'auto', 0.2], 
            'epsilon': uniform(0.01, 3),
            'C': uniform(0.1, 5)}


In [35]:
rs_svm = RandomizedSearchCV(svm, svm_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [36]:
rs_svm.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=SVR(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F525940>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F51E220>,
                                        'gamma': ['scale', 'auto', 0.2],
                                        'kernel': ['rbf', 'linear', 'poly']},
                   scoring='neg_mean_squared_error', verbose=1)

In [37]:
# Multi-Layer Perceptron
# Typically 1-2 hidden layers are adequate, and the optimal size of the first hidden layer is usually... 
# between that of the input and the output layers, or 42 and 1 in this case (42 is the max with all the time dummies)

mlp = MLPRegressor(early_stopping=True, max_iter=10000, random_state=11)

mlp_dist = {'hidden_layer_sizes': [(18,), (24,), (32,), (18,10), (24,10), (24,18), (32,10), (32,18), (32,24)],
            'alpha': uniform(0.01, 5),
            'activation': ['relu', 'tanh'],
            'solver': ['lbfgs', 'adam']}


In [38]:
rs_mlp = RandomizedSearchCV(mlp, mlp_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [39]:
rs_mlp.fit(X_cyc, y_cyc)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=MLPRegressor(early_stopping=True, max_iter=10000,
                                          random_state=11),
                   n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002074F51E7C0>,
                                        'hidden_layer_sizes': [(18,), (24,),
                                                               (32,), (18, 10),
                                                               (24, 10),
                                                               (24, 18),
                                                               (32, 10),
                                                               (32, 18),
                                                               (32, 24)],
                

### Best parameters found

In [40]:
rs_las.best_params_

{'alpha': 0.03996303581774219, 'fit_intercept': 0}

In [41]:
rs_rdg.best_params_

{'alpha': 0.20200994143604178, 'fit_intercept': 1, 'solver': 'lsqr'}

In [42]:
rs_rf.best_params_

{'max_features': 'auto', 'min_samples_split': 7, 'n_estimators': 384}

In [43]:
rs_xgb.best_params_

{'eta': 0.12747021661576857,
 'gamma': 27,
 'n_estimators': 275,
 'subsample': 0.7}

In [44]:
rs_svm.best_params_

{'C': 3.8460460356746884,
 'epsilon': 1.3873288601126184,
 'gamma': 'scale',
 'kernel': 'linear'}

In [45]:
rs_mlp.best_params_

{'activation': 'relu',
 'alpha': 1.6823481769437842,
 'hidden_layer_sizes': (18,),
 'solver': 'lbfgs'}

# Cross-Validation & Scoring on Cyclic Data

### Cross-validation on training data

In [46]:
las_scores_cyc = cross_val_score(rs_las, X_cyc, y_cyc, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [47]:
rdg_scores_cyc = cross_val_score(rs_rdg, X_cyc, y_cyc, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [48]:
rf_scores_cyc = cross_val_score(rs_rf, X_cyc, y_cyc, cv = time_split, n_jobs=-1, 
                                scoring = 'neg_mean_absolute_error')

In [49]:
xgb_scores_cyc = cross_val_score(rs_xgb, X_cyc, y_cyc, cv = time_split, 
                                 scoring = 'neg_mean_absolute_error')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [50]:
svm_scores_cyc = cross_val_score(rs_svm, X_cyc, y_cyc, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [51]:
mlp_scores_cyc = cross_val_score(rs_mlp, X_cyc, y_cyc, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [52]:
cv_dict_cyc = {
    'Lasso Regression': -np.round(las_scores_cyc.mean(), 4),
    'Ridge Regression': -np.round(rdg_scores_cyc.mean(), 4),
    'Random Forest': -np.round(rf_scores_cyc.mean(), 4),
    'Xtreme Gradient Boost': -np.round(xgb_scores_cyc.mean(), 4),
    'Support Vector Machine': -np.round(svm_scores_cyc.mean(), 4),
    'Multi-Layer Perceptron': -np.round(mlp_scores_cyc.mean(), 4),
}

In [53]:
df_cv_cyc = pd.DataFrame({'Model': cv_dict_cyc.keys(), 'Average MAE': cv_dict_cyc.values()})

### Scoring test data

In [54]:
def reg_scoring(X, y, reg_dict):
    '''
    Objective: Cycles through a dictionary of trained models, using them to make predictions, scores those 
    predictions on MAE, MSE & RMSE, and generates DataFrames of the scores and model predictions respectively
    
    X: DataFrame containing the explanatory variables
    
    y: Target variable
    
    reg_dict: Dictionary of trained/fitted models
    '''
    
    test1_scores = []
    test2_scores = []
    
    df_pred = pd.DataFrame(columns=reg_dict.keys()) # Columns of DF will accord with reg_dict keys
    
    # Loop through Dictionary items
    for key, reg in reg_dict.items():
        
        pred_y = reg.predict(X)
        df_pred[key] = pd.Series(pred_y).transpose()
        
        # Computing test scores for each model
        test1_scores.append(round(mean_absolute_error(y, pred_y), 4))
        test2_scores.append(round(mean_squared_error(y, pred_y, squared=False), 4))
        
    # Generate results DataFrame
    results = pd.DataFrame({'Model': list(reg_dict.keys()), 
                            'Mean Absolute Error': test1_scores,
                            'Root Mean Squared Error': test2_scores
                            })
    
    # Add target variable to the DataFrame of predictions
    df_pred['Target'] = y.tolist()
    
    return results, df_pred


In [55]:
# Dictionary of TRAINED models
reg_dict = {
    'Lasso Regression': rs_las,
    'Ridge Regression': rs_rdg,
    'Random Forest': rs_rf,
    'Xtreme Gradient Boost': rs_xgb,
    'Support Vector Machine': rs_svm,
    'Multi-Layer Perceptron': rs_mlp
}

In [56]:
scores_cyc, df_pred_cyc = reg_scoring(X_cyct, y_cyct, reg_dict)

In [57]:
df_pred_cyc['date'] = df_cyct.index

In [58]:
#df_pred_cyc.to_csv(r'C:\\bj_pred_cyc.csv', index=False)

# Scores

### Cross-validation scores

In [59]:
df_cv_cyc

,Model,Average MAE
0,Lasso Regression,13.0634
1,Ridge Regression,12.9651
2,Random Forest,13.9414
3,Xtreme Gradient Boost,13.6248
4,Support Vector Machine,13.1111
5,Multi-Layer Perceptron,13.3051


### Test data scores

In [60]:
scores_cyc

,Model,Mean Absolute Error,Root Mean Squared Error
0,Lasso Regression,12.1308,21.8470
1,Ridge Regression,12.0958,21.8041
2,Random Forest,12.6540,22.6529
3,Xtreme Gradient Boost,12.5097,22.7226
4,Support Vector Machine,11.8640,21.8534
5,Multi-Layer Perceptron,12.4462,22.4191
